# Additional path processing

Building on work of previous notebook to develop path processing classes

In [1]:
%matplotlib inline
#from pylab import rcParams
#rcParams['figure.figsize'] = 12, 8

%load_ext autoreload
%autoreload 2

%load_ext version_information
%version_information numpy, scipy, matplotlib, pandas

Software versions
Python 2.7.12 64bit [GCC 5.4.0 20160609]
IPython 5.4.1
OS Linux 4.10.0 28 generic x86_64 with Ubuntu 16.04 xenial
numpy 1.13.1
scipy 0.19.1
matplotlib 2.0.2
pandas 0.20.3
Thu Aug 10 15:47:26 2017 AEST

In [32]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import cv2
import pandas as pd
import uuid

import os
import sys

sys.path.append("../src")

np.random.seed(0)

def view(image):
    plt.figure()
    plt.axis('off')
    plt.imshow(image, cmap = cm.Greys_r)

In [3]:
def read_coordinates_file(coord_file_path):
    file_extension = coord_file_path.split('.')[-1]
    if file_extension == 'csv':
        df = pd.read_csv(coord_file_path)
    elif file_extension == 'json':
        df = pd.read_json(coord_file_path)
    else:
        print('Unknown coordinate file type')
        sys.exit(0)
        
    # turn coordinate column into x,y dictionary
    df['xy'] = df['xy'].apply(lambda x: eval(str(x)))

    return df

In [4]:
df = read_coordinates_file('/home/jack/Data/Caffeine_Unique_Tags/json/2017-02-14_23-32-00.json')
df = df.sort_values('frame_nums', ascending=True)
df.head(n=10)

,bee_id,classifications,flattened_28x28_tag_matrices,frame_nums,xy
0,0,47,"[29, 31, 33, 35, 36, 36, 36, 43, 48, 51, 56, 6...",0,"{u'y': 1317.45056152, u'x': 1767.85913086}"
25950,2,56,"[4, 11, 16, 17, 19, 21, 26, 28, 31, 31, 36, 36...",0,"{u'y': 975.152160644, u'x': 1351.9621582}"
31893,3,54,"[14, 14, 14, 17, 24, 29, 33, 40, 45, 49, 54, 5...",0,"{u'y': 830.224731445, u'x': 1563.7980957}"
13501,1,48,"[25, 25, 23, 24, 29, 30, 34, 38, 45, 41, 33, 3...",0,"{u'y': 1026.18273926, u'x': 1672.51904297}"
1,0,38,"[26, 26, 26, 26, 27, 27, 30, 36, 36, 36, 46, 6...",1,"{u'y': 1314.23205566, u'x': 1770.20056152}"
25951,2,56,"[23, 27, 27, 30, 31, 33, 33, 32, 32, 31, 30, 3...",1,"{u'y': 972.267761231, u'x': 1351.22009277}"
31894,3,54,"[29, 29, 26, 24, 19, 25, 32, 38, 40, 47, 49, 4...",1,"{u'y': 828.093566894, u'x': 1562.76062012}"
13502,1,48,"[28, 26, 26, 26, 25, 24, 23, 24, 28, 28, 26, 2...",1,"{u'y': 1023.3392334, u'x': 1673.25805664}"
13503,1,48,"[34, 33, 31, 32, 36, 37, 38, 41, 43, 46, 51, 5...",2,"{u'y': 1024.11804199, u'x': 1673.04772949}"
25952,2,56,"[12, 14, 14, 15, 17, 19, 23, 29, 31, 31, 31, 3...",2,"{u'y': 972.648376465, u'x': 1350.78503418}"


In [136]:
grouped_bee_id = df.groupby('bee_id')
first_bee_df = grouped_bee_id.get_group(4)
first_bee_df.head()

,bee_id,classifications,flattened_28x28_tag_matrices,frame_nums,xy
274537,4,3,None,3864,"{u'y': 858.426086426, u'x': 1942.32922363}"
45506,4,33,"[18, 18, 18, 21, 21, 24, 28, 26, 28, 30, 32, 2...",3865,"{u'y': 849.047180176, u'x': 1934.65686035}"
274538,4,3,None,3866,"{u'y': 847.017028809, u'x': 1934.07080078}"
45507,4,49,"[16, 17, 19, 19, 18, 17, 18, 22, 22, 23, 24, 2...",3867,"{u'y': 850.663330078, u'x': 1930.33630371}"
45508,4,48,"[13, 13, 16, 16, 22, 25, 24, 23, 20, 26, 29, 3...",3868,"{u'y': 846.811584473, u'x': 1930.3503418}"


In [137]:
UNKNOWN_CLASS = 3
MIXED_CLASS = 'mixed'
GAP_CLASS = 'gap'
MAX_FRAME_GAP_BETWEEN_PATHS = 10
NUM_GROUP_CLASSIFICATIONS = 40
MIN_NUM_CLASSIFIED_GROUP = 10
NUM_GROUPS_IN_SECTION = 5
CLASS_CONF_THRESH = 0.6

def increment_dict_key_value(class_dict, classification):
    if classification in class_dict.keys():
        class_dict[classification] += 1
    else:
        class_dict[classification] = 1
    return class_dict

def get_most_freq_class_pred(section_classifications):
    num_classifications = len(section_classifications)
    num_mixed = section_classifications.count(MIXED_CLASS)
    num_unknown = section_classifications.count(UNKNOWN_CLASS)
    num_mixed_unknown = num_mixed + num_unknown
    num_known = num_classifications - num_mixed_unknown
    
    if num_classifications < 2:
        return UNKNOWN_CLASS
    elif num_mixed > 1:
        return MIXED_CLASS
    elif num_mixed_unknown > 2:
        return UNKNOWN_CLASS
    elif num_known <= num_mixed_unknown:
        return UNKNOWN_CLASS
    else:
        section_count_dict = {MIXED_CLASS: 0, UNKNOWN_CLASS: 0}
        for classification in section_classifications:
            section_count_dict = increment_dict_key_value(section_count_dict, classification)
        
        del section_count_dict[MIXED_CLASS]
        del section_count_dict[UNKNOWN_CLASS]
        
        percent_section_count_dict = {c: float(section_count_dict[c]) / num_known for c in section_count_dict}
        most_freq_class_pred = max(section_count_dict, key=section_count_dict.get)
        
        if percent_section_count_dict[most_freq_class_pred] > 0.5:
            return most_freq_class_pred
        else:
            return MIXED_CLASS

class BeeData:
    def __init__(self, video_start_datetime, classification):
        self.video_start_datetime = video_start_datetime
        
        self.start_end_frame_num_all_paths = []
        self.list_of_all_xy_paths = []
        
        self.consensus_grouped_classifications = []
        self.num_classifications_in_group = 0
        
        self.class_counts_path = {UNKNOWN_CLASS: 0, GAP_CLASS: 0}
        self.add_classification(classification)
        
    def add_classification(self, classification):
        self.num_classifications_in_group += 1
        self.class_counts_path = increment_dict_key_value(self.class_counts_path, classification)
            
        if self.num_classifications_in_group == NUM_GROUP_CLASSIFICATIONS:
            self.identify_freq_class_path_group()
            self.num_classifications_in_group = 0
    
    def identify_freq_class_path_group(self):
        num_unknown_gaps_classified = self.class_counts_path[UNKNOWN_CLASS] + self.class_counts_path[GAP_CLASS]
        num_remaining_classified = self.num_classifications_in_group - num_unknown_gaps_classified
        if self.num_classifications_in_group < MIN_NUM_CLASSIFIED_GROUP or num_remaining_classified < MIN_NUM_CLASSIFIED_GROUP:
            self.consensus_grouped_classifications.append(UNKNOWN_CLASS)
        else:
            del self.class_counts_path[UNKNOWN_CLASS]
            del self.class_counts_path[GAP_CLASS]
            percent_class_counts_path = {c: float(self.class_counts_path[c]) / num_remaining_classified for c in self.class_counts_path}
            most_freq_class_pred = max(percent_class_counts_path, key=percent_class_counts_path.get)
            if percent_class_counts_path[most_freq_class_pred] > CLASS_CONF_THRESH:
                self.consensus_grouped_classifications.append(most_freq_class_pred)
            else:
                self.consensus_grouped_classifications.append(MIXED_CLASS)
        
        self.current_group_tags = []
        self.class_counts_path = {UNKNOWN_CLASS: 0, GAP_CLASS: 0}
        
    def identify_uncertain_region(self, list_classifications):
        is_uncertain_region = False
        num_mixed = list_classifications.count(MIXED_CLASS)
        num_unknown = list_classifications.count(UNKNOWN_CLASS)
        if num_mixed > 1 or num_unknown > 3 or num_mixed + num_unknown > 3:
            is_uncertain_region = True
        return is_uncertain_region
    
    
    def get_index_prev_class(self, current_path_index, current_section_pred, classifications_list):
        while True:
            if current_section_pred != classifications_list[current_path_index]:
                return current_path_index
            else:
                current_path_index -= 1
    
    def merge_group_classifications_into_sections(self):
        num_grouped_classifications = len(self.consensus_grouped_classifications)
        num_concurrent_mixed = 0
        num_concurrent_unknown = 0
        is_unknown_path_section = False
        self.class_path_end_index = []
        self.classes_in_path = []
        prev_classification = None
        
        for i in range(0, num_grouped_classifications, NUM_GROUPS_IN_SECTION):
            current_section = self.consensus_grouped_classifications[i:i+NUM_GROUPS_IN_SECTION]
            current_section_pred = get_most_freq_class_pred(current_section)
                
            if current_section_pred == MIXED_CLASS:
                num_concurrent_mixed += 1
            elif current_section_pred == UNKNOWN_CLASS:
                num_concurrent_unknown += 1
            else:
                num_concurrent_mixed_unknown = num_concurrent_mixed + num_concurrent_unknown
                current_prev_pred_same = current_section_pred == prev_classification
                
                if current_prev_pred_same:
                    if num_concurrent_mixed > 2 or num_concurrent_mixed_unknown > 6:
                        is_unknown_path_section = True
                else:
                    if num_concurrent_mixed > 1 or num_concurrent_mixed_unknown > 3:
                        is_unknown_path_section = True
                
                if is_unknown_path_section and prev_classification is None:
                    prev_classification = UNKNOWN_CLASS
                
                if prev_classification is None:
                    prev_classification = current_section_pred
                    current_prev_pred_same = True
                
                if not current_prev_pred_same or is_unknown_path_section:
                    # covers case where there's gap or new class
                    prev_class_index = self.get_index_prev_class(i,  current_section_pred, self.consensus_grouped_classifications)
                    self.class_path_end_index.append(prev_class_index * NUM_GROUP_CLASSIFICATIONS)
                    if is_unknown_path_section:
                        self.classes_in_path.append(UNKNOWN_CLASS)
                    else:
                        self.classes_in_path.append(prev_classification)
                
                prev_classification = current_section_pred
                num_concurrent_mixed = 0
                num_concurrent_unknown = 0
                is_unknown_path_section = False
        
        num_concurrent_mixed_unknown = num_concurrent_mixed + num_concurrent_unknown
        if prev_classification is None:
            self.classes_in_path.append(UNKNOWN_CLASS)
            self.class_path_end_index.append(num_grouped_classifications * NUM_GROUP_CLASSIFICATIONS)
        elif num_concurrent_mixed_unknown == 0 or num_concurrent_mixed < 3 and num_concurrent_mixed_unknown < 7:
            self.classes_in_path.append(prev_classification)
            self.class_path_end_index.append(num_grouped_classifications * NUM_GROUP_CLASSIFICATIONS)
        else:
            final_section = self.consensus_grouped_classifications[num_grouped_classifications-(num_concurrent_mixed_unknown*NUM_GROUPS_IN_SECTION):num_grouped_classifications]
            current_section_pred = get_most_freq_class_pred(final_section)
            prev_class_index = self.get_index_prev_class(num_grouped_classifications-1,  current_section_pred, self.consensus_grouped_classifications)
            self.class_path_end_index.append(prev_class_index * NUM_GROUP_CLASSIFICATIONS)
            self.classes_in_path.append(prev_classification)
            
            self.class_path_end_index.append(num_grouped_classifications * NUM_GROUP_CLASSIFICATIONS)
            self.classes_in_path.append(UNKNOWN_CLASS)
            
    def gen_separate_tag_class_bees(self):
        num_coords = 0
        path_class_index = 0
        bee_paths_index_starts = 0
        bee_tags_in_path = []
        
        class_path_coords = None
        class_path_start_end_frame_num = None
        
        list_class_path_coords = []
        list_class_path_start_end_frame_nums = []
        
        for paths_index in range(len(self.list_of_all_xy_paths)):
            class_path_coords = self.list_of_all_xy_paths[paths_index]
            class_path_start_end_frame_num = self.start_end_frame_num_all_paths[paths_index]
            for xy_index in range(len(self.list_of_all_xy_paths[paths_index])):
                
                if num_coords > self.class_path_end_index[path_class_index]:
                    list_class_path_coords.append(class_path_coords[:xy_index])
                    list_class_path_start_end_frame_nums.append({'start': class_path_start_end_frame_num['start'], 'end': class_path_start_end_frame_num['start']+xy_index-1})
                    bee_tag_data = {'video_start_datetime': self.video_start_datetime, 'bee_id': uuid.uuid4().hex, 'tag_class': self.classes_in_path[path_class_index], 'xy_paths': list_class_path_coords, 'start_end_frame_nums': list_class_path_start_end_frame_nums}
                    bee_tags_in_path.append(bee_tag_data)
                    
                    
                    class_path_start_end_frame_num['start'] += xy_index
                    class_path_coords = class_path_coords[xy_index:]
                    list_class_path_coords = []
                    list_class_path_start_end_frame_nums = []
                    
                    path_class_index += 1
                
                num_coords += 1
                
            list_class_path_coords.append(class_path_coords)
            list_class_path_start_end_frame_nums.append(class_path_start_end_frame_num)
                
        bee_tag_data = {'video_start_datetime': self.video_start_datetime, 'bee_id': uuid.uuid4().hex, 'tag_class': self.classes_in_path[path_class_index], 'xy_paths': list_class_path_coords, 'start_end_frame_nums': list_class_path_start_end_frame_nums}
        bee_tags_in_path.append(bee_tag_data)
        
        return bee_tags_in_path

In [138]:
class ProcessPaths:

    def __init__(self, video_start_datetime):
        self.video_start_datetime = video_start_datetime

    def gen_gap_coords(self, current_xy_coord, prev_xy_coord, difference_prev_frame):
        x1, y1 = current_xy_coord['x'], current_xy_coord['y']
        x2, y2 = prev_xy_coord['x'], prev_xy_coord['y']
        x_diff_per_frame = (x2 - x1) / float(difference_prev_frame)
        y_diff_per_frame = (y2 - y1) / float(difference_prev_frame)

        gap_coords = []
        for gap in range(1, difference_prev_frame + 1):
            x_gap_coord = x2 - x_diff_per_frame * gap
            y_gap_coord = y2 - y_diff_per_frame * gap
            gap_coords.append({'x': x_gap_coord, 'y': y_gap_coord})

        return gap_coords

    def process_paths(self, bee_df):
        xy_list = bee_df['xy'].tolist()
        frame_nums_list = bee_df['frame_nums'].tolist()
        classifications_list = bee_df['classifications'].tolist()
        #flat_tags = bee_df['flattened_28x28_tag_matrices'].tolist()

        bee_data = BeeData(self.video_start_datetime, classifications_list[0])
        start_end_frame_num_path = {'start': frame_nums_list[0],'end': frame_nums_list[0]}
        xy_path = [xy_list[0]]
        
        for i in range(1, len(xy_list)):
            difference_prev_frame = frame_nums_list[i] - start_end_frame_num_path['end']

            if difference_prev_frame == 1:
                start_end_frame_num_path['end'] = frame_nums_list[i]
                xy_path.append(xy_list[i])
                bee_data.add_classification(classifications_list[i])
                
            elif difference_prev_frame < MAX_FRAME_GAP_BETWEEN_PATHS:
                start_end_frame_num_path['end'] = frame_nums_list[i]
                generated_coord_gaps = self.gen_gap_coords(xy_list[i], xy_list[i-1], difference_prev_frame)
                fill_path_classifications_gap = ['gap'] * len(generated_coord_gaps)
                fill_path_classifications_gap[-1] = classifications_list[i]
                xy_path.extend(generated_coord_gaps)
        
                for gap_classification in fill_path_classifications_gap:
                    bee_data.add_classification(gap_classification)
            else:
                bee_data.list_of_all_xy_paths.append(xy_path)
                bee_data.start_end_frame_num_all_paths.append(start_end_frame_num_path)
                xy_path = [xy_list[i]]
                start_end_frame_num_path = {'start': frame_nums_list[i], 'end': frame_nums_list[i]}
                bee_data.add_classification(classifications_list[i])

        if len(xy_path) > 0:
            bee_data.list_of_all_xy_paths.append(xy_path)
            bee_data.start_end_frame_num_all_paths.append(start_end_frame_num_path)
            bee_data.identify_freq_class_path_group()

        bee_data.merge_group_classifications_into_sections()
        
        #bee_data.class_path_end_index = [100, 5000, 13880]
        #bee_data.classes_in_path = [1, 2, 38]
        
        #print(bee_data.class_path_end_index)
        #print(bee_data.classes_in_path)
        
        bees_identified_by_tag = bee_data.gen_separate_tag_class_bees()
        
        return bees_identified_by_tag

In [139]:
pp = ProcessPaths('2017-02-14_23-32-00')
bd = pp.process_paths(first_bee_df)
#print(len(bd['xy_paths']), len(bd['xy_paths'][0]), len(bd['xy_paths'][1]))
#del bd['xy_paths']

for b in bd:
    for i in range(len(b['xy_paths'])):
        print(b['start_end_frame_nums'][i]['end']-b['start_end_frame_nums'][i]['start'], len(b['xy_paths'][i]))
    del b['xy_paths']
    print(b)

(132, 133)
{'tag_class': 3, 'bee_id': 'fa447f5b65b74ba1a7d2e966d261723f', 'video_start_datetime': '2017-02-14_23-32-00', 'start_end_frame_nums': [{'start': 3864, 'end': 3996}]}


In [130]:
ii = 0
iii = []
for i in range(13768, 13923+1):
    ii += 1
    iii.append((ii, i))
print(iii[-1])

(156, 13923)


In [141]:
grouped_bee_id = df.groupby('bee_id')
for group_name, group_df in grouped_bee_id:
    pp = ProcessPaths('2017-02-14_23-32-00')
    bd = pp.process_paths(group_df)
    
    for b in bd:
        del b['xy_paths']
        print(b)

{'tag_class': 38, 'bee_id': 'cad2e65b554342139e16f3ee8fea38fe', 'video_start_datetime': '2017-02-14_23-32-00', 'start_end_frame_nums': [{'start': 0, 'end': 13713}, {'start': 13768, 'end': 13923}]}
{'tag_class': 48, 'bee_id': 'fdc5a59cd7b84ade9f12d8c054429fee', 'video_start_datetime': '2017-02-14_23-32-00', 'start_end_frame_nums': [{'start': 0, 'end': 14744}]}
{'tag_class': 56, 'bee_id': '2288a3f728054748b91d5f3aae9360f6', 'video_start_datetime': '2017-02-14_23-32-00', 'start_end_frame_nums': [{'start': 0, 'end': 6287}, {'start': 6297, 'end': 6515}, {'start': 6533, 'end': 7360}, {'start': 7373, 'end': 10888}]}
{'tag_class': 54, 'bee_id': '745ed863661b469b9a4172a7f5f8072b', 'video_start_datetime': '2017-02-14_23-32-00', 'start_end_frame_nums': [{'start': 0, 'end': 16028}]}
{'tag_class': 3, 'bee_id': '50519160a276474ca103722f7334f1bf', 'video_start_datetime': '2017-02-14_23-32-00', 'start_end_frame_nums': [{'start': 3864, 'end': 3996}]}
{'tag_class': 3, 'bee_id': '942ecda517804eb5b461ea2c

{'tag_class': 5, 'bee_id': '910190b0090b40a7b21c564304041d9c', 'video_start_datetime': '2017-02-14_23-32-00', 'start_end_frame_nums': [{'start': 37729, 'end': 38449}]}
{'tag_class': 54, 'bee_id': '9a38891112f9436a95a8f2f185a94b91', 'video_start_datetime': '2017-02-14_23-32-00', 'start_end_frame_nums': [{'start': 38450, 'end': 40163}]}
{'tag_class': 3, 'bee_id': '252a7d7aef3b4e7b8a5715f0a5904c22', 'video_start_datetime': '2017-02-14_23-32-00', 'start_end_frame_nums': [{'start': 38211, 'end': 38801}]}
{'tag_class': 5, 'bee_id': '7b962774469f4229b2401acf89e35383', 'video_start_datetime': '2017-02-14_23-32-00', 'start_end_frame_nums': [{'start': 38497, 'end': 38722}, {'start': 38756, 'end': 38792}, {'start': 38831, 'end': 38940}]}
{'tag_class': 3, 'bee_id': 'b82b474a9e1c4e22b94516c067c20674', 'video_start_datetime': '2017-02-14_23-32-00', 'start_end_frame_nums': [{'start': 38777, 'end': 38802}]}
{'tag_class': 3, 'bee_id': '7c031518031a484382ad90805bdf0ed9', 'video_start_datetime': '2017-02

{'tag_class': 3, 'bee_id': 'c607cae419784de6a817056943614af4', 'video_start_datetime': '2017-02-14_23-32-00', 'start_end_frame_nums': [{'start': 68809, 'end': 68812}]}
{'tag_class': 3, 'bee_id': '9fa0769fd8c94206b8484d372788c6dd', 'video_start_datetime': '2017-02-14_23-32-00', 'start_end_frame_nums': [{'start': 68839, 'end': 68840}]}
{'tag_class': 3, 'bee_id': 'e0649b65ed9a4ce5bce5fb9a3e38bd0d', 'video_start_datetime': '2017-02-14_23-32-00', 'start_end_frame_nums': [{'start': 69794, 'end': 69802}]}
{'tag_class': 32, 'bee_id': '7fc013b70e7944fa9af4bd84eeff61a4', 'video_start_datetime': '2017-02-14_23-32-00', 'start_end_frame_nums': [{'start': 70235, 'end': 71992}]}
{'tag_class': 3, 'bee_id': '4027ce1cf35b4a1d87aab93c341aa42e', 'video_start_datetime': '2017-02-14_23-32-00', 'start_end_frame_nums': [{'start': 70295, 'end': 70308}]}
{'tag_class': 3, 'bee_id': '428956aa92dc4d03b904247efa866bbf', 'video_start_datetime': '2017-02-14_23-32-00', 'start_end_frame_nums': [{'start': 70341, 'end': 